# Abstrakcija

## Moduli

"Ena škatla, v katero lahko naberemo stvari not", "zbirka definicij"
Npr. List., Array.,... 

Vsaka .ml datoteka je module. in če ji želimo napisati še signaturo naredimo posebej file ime_datoteke.mli (to je tist gumbek ml/mli). Načeloma se prof temu ne bo posluževal, ker je pisanje v eni datoteki močnejše in nam omogoča gnezdenje signatur in modulov.

In [1]:
type datum = { dan : int; mesec : int; leto : int }

let je_prestopno leto =
  (leto mod 4 = 0 && leto mod 100 <> 0) || leto mod 400 = 0
  
let dolzina_meseca leto =
  function
  | 4 | 6 | 9 | 11 -> 30
  | 2 -> if je_prestopno leto then 29 else 28
  | _ -> 31

let je_veljaven datum =
  let veljaven_dan = 1 <= datum.dan && datum.dan <= dolzina_meseca datum.leto datum.mesec
  and veljaven_mesec = 1 <= datum.mesec && datum.mesec <= 12
  in
  veljaven_dan && veljaven_mesec

let naredi_datum dan mesec leto =
  let datum = { dan; mesec; leto } in
  if je_veljaven datum then Some datum else None

let to_string { dan; mesec; leto } =
  Format.sprintf "%04d-%02d-%02d" leto mesec dan

type datum = { dan : int; mesec : int; leto : int; }


val je_prestopno : int -> bool = <fun>


val dolzina_meseca : int -> int -> int = <fun>


val je_veljaven : datum -> bool = <fun>


val naredi_datum : int -> int -> int -> datum option = <fun>


val to_string : datum -> string = <fun>


Definiramo Module

In [1]:
module Datum = struct
  type t = { dan : int; mesec : int; leto : int }

  let je_prestopno leto =
    (leto mod 4 = 0 && leto mod 100 <> 0) || leto mod 400 = 0
    
  let dolzina_meseca leto =
    function
    | 4 | 6 | 9 | 11 -> 30
    | 2 -> if je_prestopno leto then 29 else 28
    | _ -> 31

  let je_veljaven datum =
    let veljaven_dan = 1 <= datum.dan && datum.dan <= dolzina_meseca datum.leto datum.mesec
    and veljaven_mesec = 1 <= datum.mesec && datum.mesec <= 12
    in
    veljaven_dan && veljaven_mesec

  let naredi dan mesec leto =
    let datum = { dan; mesec; leto } in
    if je_veljaven datum then Some datum else None

  let to_string { dan; mesec; leto } =
    Format.sprintf "%04d-%02d-%02d" leto mesec dan
end

module Datum :
  sig
    type t = { dan : int; mesec : int; leto : int; }
    val je_prestopno : int -> bool
    val dolzina_meseca : int -> int -> int
    val je_veljaven : t -> bool
    val naredi : int -> int -> int -> t option
    val to_string : t -> string
  end


Tako kličemo funkcije iz modula, takoj, ko smo zunaj end.

In [3]:
Datum.je_prestopno 2024

- : bool = true


Modul lahko odpreš (open Datum...podoben kt from_ import_ )... tega ne uporabljat, ker se ti lahko pokvari npr. na nov definiran +. Raje odpreš lokalno z let in znotraj funkcije.

In [2]:
let open Datum in
je_veljaven { dan = 26; mesec = 6; leto = 1991 }  

- : bool = true


```
let _ =
let open Datum in
let x ...
```
ali
```
let _ = Polinom. (...)
```

In [3]:
module D = Datum
let je_veljaven = Datum.je_veljaven

module D = Datum


In [5]:
type narocilo = {
  ime : string;
  datum_narocila : Datum.t;
}

type narocilo = { ime : string; datum_narocila : Datum.t; }


## Signature

### Definicije signatur

Znotrej modula ti za vsako funkcijo pove kam se iz kam slika. to mamo lahko kot output v terminalu, ali pa svoje napišemo in jih lahko spremenimo. Navada je da so all caps.

Tudi tukaj jih lahko preimenujemo (tako kot mreza v dn)

In [5]:
module type DATUM = sig
  type t = { dan : int; mesec : int; leto : int; }
  val je_prestopno : int -> bool
  val dolzina_meseca : int -> int -> int
  val je_veljaven : t -> bool
  val naredi : int -> int -> int -> t option
  val to_string : t -> string
end

module type DATUM =
  sig
    type t = { dan : int; mesec : int; leto : int; }
    val je_prestopno : int -> bool
    val dolzina_meseca : int -> int -> int
    val je_veljaven : t -> bool
    val naredi : int -> int -> int -> t option
    val to_string : t -> string
  end


### Preverjanje signatur

Lahko jo spišemo in rečemo OCaml da preveri, če vse funkciije zadoščajo te signaturi.
Z ```:``` povemo, da more določen Module zadoščati določeni Signaturi.

Glavni nameni signature:
- preverja a je vse notr
- daje vpogled v to kaj je notr
- (glavna stvar:) določa da skrijemo stvari

"Špageti koda"...dejanski izraz, ker je vse prepleten v eni funkciji kličeš drugo, v drugi tretjo,...

Apparantly hočeš vse pomožne funkcije skrit (da jih ne uporablaš večkrat hah). Signatura nam predstavi vse ki so vidne.

s tem ko kličemo DATUM. nam prikaže vse funckije ki imamo za to

In [9]:
module Datum : DATUM = struct
  type t = { dan : int; mesec : int; leto : int }

  let je_prestopno leto =
    (leto mod 4 = 0 && leto mod 100 <> 0) || leto mod 400 = 0
    
  let dolzina_meseca leto =
    function
    | 4 | 6 | 9 | 11 -> 30
    | 2 -> if je_prestopno leto then 29 else 28
    | _ -> 31

  let je_veljaven datum =
    let veljaven_dan = 1 <= datum.dan && datum.dan <= dolzina_meseca datum.leto datum.mesec
    and veljaven_mesec = 1 <= datum.mesec && datum.mesec <= 12
    in
    veljaven_dan && veljaven_mesec

  let naredi dan mesec leto =
    let datum = { dan; mesec; leto } in
    if je_veljaven datum then Some datum else None

  let to_string { dan; mesec; leto } =
    Format.sprintf "%04d-%02d-%02d" leto mesec dan
end

module Datum : DATUM


In [11]:
module Datum : DATUM = struct
  type t = { dan : int; mesec : int; leto : int }

  let je_prestopno _ = true
    
  let dolzina_meseca _ _ = 30

  let je_veljaven _ = true

  let naredi dan mesec leto = None

  let to_string { dan; mesec; leto } = "dan"
end

module Datum : DATUM


### Skrivanje vrednosti

Opis kode lahko naredimo tudi znotraj signature in sicer (** opis kode, kjer vsa koda izgleda kot [koda] *)

In [26]:
module type ULOMEK = sig
  type t = { stevec : int; imenovalec : int; }
  val ( * ) : t -> t -> t
  val ( + ) : t -> t -> t
end

module type ULOMEK =
  sig
    type t = { stevec : int; imenovalec : int; }
    val ( * ) : t -> t -> t
    val ( + ) : t -> t -> t
  end


In [27]:
module Ulomek : ULOMEK = struct
  type t = { stevec : int; imenovalec : int }
  let rec gcd a b = if b = 0 then a else gcd b (a mod b)
  let okrajsaj { stevec; imenovalec } =
    let delitelj = gcd stevec imenovalec in
    { stevec = stevec / delitelj; imenovalec = imenovalec / delitelj }
  let ( *** ) p q = { stevec = p.stevec * q.stevec; imenovalec = p.imenovalec * q.imenovalec } |> okrajsaj
  let ( +++ ) p q = { stevec = p.stevec * q.imenovalec + q.stevec * p.imenovalec; imenovalec = p.imenovalec * q.imenovalec } |> okrajsaj
  let ( * ) = ( *** )
  let ( + ) = ( +++ )
end

module Ulomek : ULOMEK


In [32]:
Ulomek.{ stevec = 10; imenovalec = 100}

- : Ulomek.t = {Ulomek.stevec = 10; imenovalec = 100}


In [29]:
let a = Ulomek.{ stevec = 1; imenovalec = 2 }
let b = Ulomek.{ stevec = 1; imenovalec = 3 }
let c = Ulomek.(a + b * a + a * b)

val a : Ulomek.t = {Ulomek.stevec = 1; imenovalec = 2}


val b : Ulomek.t = {Ulomek.stevec = 1; imenovalec = 3}


val c : Ulomek.t = {Ulomek.stevec = 5; imenovalec = 6}


In [30]:
module Ulomek : ULOMEK = struct
  type t = int * int
  let rec gcd a b = if b = 0 then a else gcd b (a mod b)
  let okrajsaj (p, q) = let d = gcd p q in (p / d,  q / d )
  let ( *** ) (p1, p2) (q1, q2) = (p1 * q1, p2 * q2) |> okrajsaj
  let ( +++ ) (p1, p2) (q1, q2) = (p1 * q2 + q1 * p2, p2 * q2) |> okrajsaj
  let ( * ) = ( *** )
  let ( + ) = ( +++ )
end

error: compile_error

### Abstrakcija tipov

Abstraktni tipi so tipi, kjer ne vemo kaj je notri. Če tako napišemo, tud OCaml ne bo enačil med našim napisanim tipom in standrdnemu zapisu tipa npr t = list, OCaml se bo pritožu da hoče list ima pa t. Lahko pa dodamo trivialne funkcije (fun x -> x), ki jim določimo tip t -> list in list -> t, da jih spravimo v pravi tip.

neka korelacija z wrapper v pythonu.

In [45]:
module type ULOMEK = sig
  type t
  val of_int : int -> t
  val ( // ) : int -> int -> t
  val ( * ) : t -> t -> t
  val ( / ) : t -> t -> t
  val ( + ) : t -> t -> t
  val to_string : t -> string
end

module type ULOMEK =
  sig
    type t
    val of_int : int -> t
    val ( // ) : int -> int -> t
    val ( * ) : t -> t -> t
    val ( / ) : t -> t -> t
    val ( + ) : t -> t -> t
    val to_string : t -> string
  end


In [46]:
module Ulomek : ULOMEK = struct
  type t = int * int
  let of_int x = (x, 1)
  let rec gcd a b = if b = 0 then a else gcd b (a mod b)
  let okrajsaj (p, q) = let d = gcd p q in (p / d,  q / d )
  let ( // ) p q = okrajsaj (p, q)
  let ( *** ) (p1, p2) (q1, q2) = (p1 * q1, p2 * q2) |> okrajsaj
  let ( +++ ) (p1, p2) (q1, q2) = (p1 * q2 + q1 * p2, p2 * q2) |> okrajsaj
  let ( / ) (p1, p2) (q1, q2) = (p1 * q2, p2 * q1) |> okrajsaj
  let ( * ) = ( *** )
  let ( + ) = ( +++ )
  let to_string (p, q) = Format.sprintf "%d/%d" p q
end

module Ulomek : ULOMEK


In [50]:
Ulomek.(1 // 2 + 1 // 3 |> to_string) |> print_endline

- : unit = ()


5/6


# Neobvezen, ampak vseen uporaben del

vse te svoje tipe smo jim dodali alfa odspredej in smo šli popravljat funkcije. ampak to menda ni dejanski način in bi rabili module, da lahko zapakiramo skupej eno funkcijo in nekej stvari zraven.

In [ ]:
module type ULOMEK = sig
  type t
  val of_alfa : 'a -> 'a
  val ( // ) : 'a -> 'a -> 'a t
  val ( * ) : 'a t -> 'a t -> 'a t
  val ( / ) : 'a t -> 'a t -> 'a t
  val ( + ) : 'a t -> 'a t -> 'a t
  val to_string : 'a t -> string
end

module type ULOMEK =
  sig
    type t
    val of_alfa : 'a -> 'a
    val ( // ) : 'a -> 'a -> t
    val ( * ) : t -> t -> t
    val ( / ) : t -> t -> t
    val ( + ) : t -> t -> t
    val to_string : t -> string
  end


Rabimo definirati kolobar (iz algebre)

elt je krajše za element

In [3]:
module type KOLOBAR = sig
  type elt

  let ( + ) = ( + ) 
  let ( * ) = ( * )

end

module type POLINOM = sig
  type polinom
  type kolobar

end

module celostevilski_polinom : POLINOM
  type polinom

end

module Polinom_nad_kolobarjem (K : KOLOBAR)  = struct
  (* enter a lieu of sighs ker nekej ne dela*)
  type polinom = K.elt list
  type kolobar = K.elt

  ...

end

error: compile_error

Lahko gnezdimo module.
Npr. mamo modul ki implementira števila in ga rabimo za implementacijo modulov, ki definirajo polinome. 

Modulom, ki vzamejo modul in druge stvari imenujemo funktorji

### Mal bl random shit

Za 2. dn bomo morali znati uporabiti učinkovit slovar, ki je trenutno v knjižnici Map.OrderedType

In [4]:
module NiziSPrimerjavo = Map.OrderedType = struct
  type t = string

  let compare = String.compare (* ali let compare = String.compare in bil v zadnjem modulu*)
end

module SlovarZNiziZaKljuce = Map.Make(NiziSPrimerjavo)

  let _ =
    SlovarZNiziZaKljuce.empty 
    |> SlovarZNiziZaKljuce.add "kljuc" 10

error: compile_error